In [10]:
from utils.data_loader import load_data
from utils.feature_engineering import add_date_features
from utils.evaluation import evaluate_predictions
from utils.plot_utils import plot_predictions, plot_relative_error
from model.tide_model import TiDEWithEmbedding
from model.train import prepare_datasets, train_model
import pandas as pd
import numpy as np
import torch

def process_label(df, label_col, diff_col, target):
    df[target] = df[label_col] - df[diff_col]
    return df




# 参数
FILEPATH = './data/ad_install_ltv_sequence_feature.obj'
DATE_CUTOFF = '2025-05-10'
promo_dates = [

    {"date": "2022-02-01", "festival": "春节"},
    {"date": "2022-02-14", "festival": "情人节"},
    {"date": "2022-03-08", "festival": "女王节"},
    {"date": "2022-05-20", "festival": "520表白节"},
    {"date": "2022-06-18", "festival": "618年中大促"},
    {"date": "2022-08-04", "festival": "七夕节"},
    {"date": "2022-09-10", "festival": "中秋节"},
    {"date": "2022-10-01", "festival": "国庆大促"},
    {"date": "2022-11-11", "festival": "双11"},
    {"date": "2022-12-12", "festival": "双12"},
    {"date": "2022-12-25", "festival": "圣诞节"},

    {"date": "2023-01-22", "festival": "春节"},
    {"date": "2023-02-14", "festival": "情人节"},
    {"date": "2023-03-08", "festival": "女王节"},

    {"date": "2023-05-20", "festival": "520表白节"},
    {"date": "2023-06-18", "festival": "618年中大促"},
    {"date": "2023-08-22", "festival": "七夕节"},
    {"date": "2023-09-29", "festival": "中秋节"},
    {"date": "2023-10-01", "festival": "国庆大促"},
    {"date": "2023-11-11", "festival": "双11"},
    {"date": "2023-12-12", "festival": "双12"},
    {"date": "2023-12-25", "festival": "圣诞节"},

    {"date": "2024-02-10", "festival": "春节"},
    {"date": "2024-02-14", "festival": "情人节"},
    {"date": "2024-03-08", "festival": "女王节"},

    {"date": "2024-05-20", "festival": "520表白节"},
    {"date": "2024-06-18", "festival": "618年中大促"},
    {"date": "2024-08-10", "festival": "七夕节"},
    {"date": "2024-09-17", "festival": "中秋节"},
    {"date": "2024-10-01", "festival": "国庆大促"},
    {"date": "2024-11-11", "festival": "双11"},
    {"date": "2024-12-12", "festival": "双12"},
    {"date": "2024-12-25", "festival": "圣诞节"},

    {"date": "2025-01-29", "festival": "春节"},
    {"date": "2025-02-14", "festival": "情人节"},
    {"date": "2025-03-08", "festival": "女王节"},
    {"date": "2025-05-20", "festival": "520表白节"},
    {"date": "2025-06-18", "festival": "618年中大促"},
    {"date": "2025-07-29", "festival": "七夕节"},
    {"date": "2025-10-06", "festival": "中秋节"},
    {"date": "2025-10-01", "festival": "国庆大促"},
    {"date": "2025-11-11", "festival": "双11"},
    {"date": "2025-12-12", "festival": "双12"},
    {"date": "2025-12-25", "festival": "圣诞节"},
]

PROMO_DATES = [ pdate['date'] for pdate in promo_dates if pdate['festival'] in  ('618年中大促', '双11','双12', '圣诞节')]

HOLIDAY_DATES = [
    '2022-01-01','2022-02-01',  '2022-05-01', '2022-06-03', '2022-09-10', '2022-10-01', '2022-10-04', '2022-01-31',
    '2023-01-01', '2023-01-22',  '2023-05-01', '2023-06-22', '2023-09-29', '2023-10-01', '2023-10-23', '2023-01-21',
    '2024-01-01','2024-02-10',  '2024-05-01', '2024-06-10', '2024-09-17', '2024-10-01', '2024-10-02', '2024-02-09',
    '2025-01-01','2025-01-29',  '2025-05-01', '2025-06-20', '2025-10-06', '2025-10-01', '2025-09-30', '2025-01-28'
]

# 加载数据
df = load_data(FILEPATH, DATE_CUTOFF)
df['diff_0'] = 0
df = df[df.dim_ad_channel_level1 == '信息流']

In [11]:
df[:100]

dim_ad_channel_level1  install_deviceid_cnt  day1_ltv_sum  day2_ltv_sum   
2                      信息流                 27353    507017.000    666733.230  \
6                      信息流                 31204    558389.910    736437.020   
10                     信息流                 30202    580690.450    752513.990   
14                     信息流                 33856    559693.310    756857.330   
18                     信息流                 33976    500054.140    749159.160   
...                    ...                   ...           ...           ...   
4890                   信息流                 36073   1061888.325   1366837.924   
4894                   信息流                 30986    973896.228   1215973.703   
4898                   信息流                 29514    937409.844   1158876.711   
4902                   信息流                 27344    927233.167   1112918.444   
4906                   信息流                 27660    907198.518   1083456.099   

      day3_ltv_sum  day4_ltv_sum  day5_ltv_sum  day6_ltv_sum  day7_ltv_sum   
2       748206.320    815981.510    874111.480    913893.320    963841.310  \
6       835375.870    908750.340    969025.390   1019263.330   1081194.030   
10      819027.980    913092.940    983862.530   1062928.130   1129981.910   
14      889135.870   1004461.710   1116927.720   1192042.100   1269620.590   
18      870898.470   1005878.500   1074578.030   1146993.960   1186151.150   
...            ...           ...           ...           ...           ...   
4890   1495795.537   1615726.622   1683405.829   1732028.455   1801983.747   
4894   1312797.649   1380998.802   1422922.024   1466021.178   1509906.493   
4898   1256871.260   1305169.972   1349275.468   1402189.676   1444136.247   
4902   1195574.729   1286916.630   1335590.691   1390207.652   1428940.536   
4906   1170500.477   1251216.426   1297376.120   1346577.013   1388785.108   

      day30_ltv_sum  day60_ltv_sum  day90_ltv_sum  day180_ltv_sum   
2       1436548.310    1624616.130    1723192.630     1903053.960  \
6       1627451.270    1795978.750    1916883.130     2201911.050   
10      1748719.780    1918868.070    2053165.580     2319873.080   
14      1902413.680    2110504.690    2253177.060     2578037.190   
18      1758961.930    2016451.510    2158952.130     2484225.810   
...             ...            ...            ...             ...   
4890    2049220.369    2049220.369    2049220.369     2049220.369   
4894    1668047.033    1668047.033    1668047.033     1668047.033   
4898    1583973.501    1583973.501    1583973.501     1583973.501   
4902    1548267.955    1548267.955    1548267.955     1548267.955   
4906    1454887.100    1454887.100    1454887.100     1454887.100   

      day360_ltv_sum          dt  diff_0  
2        2260588.080  2021-12-31       0  
6        2697690.610  2022-01-01       0  
10       2834951.410  2022-01-02       0  
14       3123151.780  2022-01-03       0  
18       3034044.610  2022-01-04       0  
...              ...         ...     ...  
4890     2049220.369  2025-05-06       0  
4894     1668047.033  2025-05-07       0  
4898     1583973.501  2025-05-08       0  
4902     1548267.955  2025-05-09       0  
4906     1454887.100  2025-05-10       0  

[1227 rows x 16 columns]

In [13]:
df.iloc[100:200]

dim_ad_channel_level1  install_deviceid_cnt  day1_ltv_sum  day2_ltv_sum   
402                   信息流                 17784     282245.46     350603.73  \
406                   信息流                 17034     254684.13     357564.13   
410                   信息流                 13074     247808.36     308935.27   
414                   信息流                 11974     195467.04     266083.99   
418                   信息流                 11309     220599.51     304687.29   
..                    ...                   ...           ...           ...   
782                   信息流                 23184     545701.09     754751.26   
786                   信息流                 22766     552075.95     702203.86   
790                   信息流                 23129     627528.81     773112.26   
794                   信息流                 24975     638409.93     835926.96   
798                   信息流                 23210     664511.28     845715.03   

     day3_ltv_sum  day4_ltv_sum  day5_ltv_sum  day6_ltv_sum  day7_ltv_sum   
402     393595.67     417406.27     440644.67     471710.75     495978.30  \
406     397243.76     426714.67     450652.59     469478.03     479484.32   
410     337837.47     381623.88     400625.21     419754.69     432843.98   
414     304964.73     330115.97     345474.80     357288.16     364585.57   
418     343921.44     366726.71     373280.22     380128.90     396412.48   
..            ...           ...           ...           ...           ...   
782     818900.00     860907.09     898272.49     937993.00     965827.43   
786     770716.13     837217.21     869222.54     890649.63     917960.88   
790     863946.27     922929.91     965883.28    1007856.73    1063113.52   
794     915369.51     964674.88    1038485.22    1088786.96    1134277.59   
798     911334.23     956609.35    1004673.09    1042218.45    1075489.94   

     day30_ltv_sum  day60_ltv_sum  day90_ltv_sum  day180_ltv_sum   
402      741210.77     901567.520    1067988.510     1370291.070  \
406      664411.66     818766.140     953242.860     1183293.070   
410      600679.19     732546.930     856220.920     1073395.968   
414      531241.18     649828.310     755603.700      973847.030   
418      561647.70     662262.100     782214.410      977986.930   
..             ...            ...            ...             ...   
782     1337997.04    1589224.800    1788105.370     2210556.518   
786     1304139.16    1530458.760    1682912.666     2077282.929   
790     1441951.65    1668869.670    1848257.970     2308504.600   
794     1510183.75    1753110.006    1947620.526     2459562.226   
798     1438873.30    1695967.660    1880700.370     2304113.700   

     day360_ltv_sum          dt  diff_0  
402     1791686.261  2022-04-10       0  
406     1535836.501  2022-04-11       0  
410     1391592.388  2022-04-12       0  
414     1270596.380  2022-04-13       0  
418     1310634.817  2022-04-14       0  
..              ...         ...     ...  
782     2804971.884  2022-07-14       0  
786     2624023.669  2022-07-15       0  
790     2889075.890  2022-07-16       0  
794     3036513.432  2022-07-17       0  
798     2894393.400  2022-07-18       0  

[100 rows x 16 columns]

In [14]:
df[100:200]

dim_ad_channel_level1  install_deviceid_cnt  day1_ltv_sum  day2_ltv_sum   
402                   信息流                 17784     282245.46     350603.73  \
406                   信息流                 17034     254684.13     357564.13   
410                   信息流                 13074     247808.36     308935.27   
414                   信息流                 11974     195467.04     266083.99   
418                   信息流                 11309     220599.51     304687.29   
..                    ...                   ...           ...           ...   
782                   信息流                 23184     545701.09     754751.26   
786                   信息流                 22766     552075.95     702203.86   
790                   信息流                 23129     627528.81     773112.26   
794                   信息流                 24975     638409.93     835926.96   
798                   信息流                 23210     664511.28     845715.03   

     day3_ltv_sum  day4_ltv_sum  day5_ltv_sum  day6_ltv_sum  day7_ltv_sum   
402     393595.67     417406.27     440644.67     471710.75     495978.30  \
406     397243.76     426714.67     450652.59     469478.03     479484.32   
410     337837.47     381623.88     400625.21     419754.69     432843.98   
414     304964.73     330115.97     345474.80     357288.16     364585.57   
418     343921.44     366726.71     373280.22     380128.90     396412.48   
..            ...           ...           ...           ...           ...   
782     818900.00     860907.09     898272.49     937993.00     965827.43   
786     770716.13     837217.21     869222.54     890649.63     917960.88   
790     863946.27     922929.91     965883.28    1007856.73    1063113.52   
794     915369.51     964674.88    1038485.22    1088786.96    1134277.59   
798     911334.23     956609.35    1004673.09    1042218.45    1075489.94   

     day30_ltv_sum  day60_ltv_sum  day90_ltv_sum  day180_ltv_sum   
402      741210.77     901567.520    1067988.510     1370291.070  \
406      664411.66     818766.140     953242.860     1183293.070   
410      600679.19     732546.930     856220.920     1073395.968   
414      531241.18     649828.310     755603.700      973847.030   
418      561647.70     662262.100     782214.410      977986.930   
..             ...            ...            ...             ...   
782     1337997.04    1589224.800    1788105.370     2210556.518   
786     1304139.16    1530458.760    1682912.666     2077282.929   
790     1441951.65    1668869.670    1848257.970     2308504.600   
794     1510183.75    1753110.006    1947620.526     2459562.226   
798     1438873.30    1695967.660    1880700.370     2304113.700   

     day360_ltv_sum          dt  diff_0  
402     1791686.261  2022-04-10       0  
406     1535836.501  2022-04-11       0  
410     1391592.388  2022-04-12       0  
414     1270596.380  2022-04-13       0  
418     1310634.817  2022-04-14       0  
..              ...         ...     ...  
782     2804971.884  2022-07-14       0  
786     2624023.669  2022-07-15       0  
790     2889075.890  2022-07-16       0  
794     3036513.432  2022-07-17       0  
798     2894393.400  2022-07-18       0  

[100 rows x 16 columns]

In [18]:
FILEPATH = './data/ad_install_ltv_sequence_feature.obj'
DATE_CUTOFF = '2025-05-10'
promo_dates = [

    {"date": "2022-02-01", "festival": "春节"},
    {"date": "2022-02-14", "festival": "情人节"},
    {"date": "2022-03-08", "festival": "女王节"},
    {"date": "2022-05-20", "festival": "520表白节"},
    {"date": "2022-06-18", "festival": "618年中大促"},
    {"date": "2022-08-04", "festival": "七夕节"},
    {"date": "2022-09-10", "festival": "中秋节"},
    {"date": "2022-10-01", "festival": "国庆大促"},
    {"date": "2022-11-11", "festival": "双11"},
    {"date": "2022-12-12", "festival": "双12"},
    {"date": "2022-12-25", "festival": "圣诞节"},

    {"date": "2023-01-22", "festival": "春节"},
    {"date": "2023-02-14", "festival": "情人节"},
    {"date": "2023-03-08", "festival": "女王节"},

    {"date": "2023-05-20", "festival": "520表白节"},
    {"date": "2023-06-18", "festival": "618年中大促"},
    {"date": "2023-08-22", "festival": "七夕节"},
    {"date": "2023-09-29", "festival": "中秋节"},
    {"date": "2023-10-01", "festival": "国庆大促"},
    {"date": "2023-11-11", "festival": "双11"},
    {"date": "2023-12-12", "festival": "双12"},
    {"date": "2023-12-25", "festival": "圣诞节"},

    {"date": "2024-02-10", "festival": "春节"},
    {"date": "2024-02-14", "festival": "情人节"},
    {"date": "2024-03-08", "festival": "女王节"},

    {"date": "2024-05-20", "festival": "520表白节"},
    {"date": "2024-06-18", "festival": "618年中大促"},
    {"date": "2024-08-10", "festival": "七夕节"},
    {"date": "2024-09-17", "festival": "中秋节"},
    {"date": "2024-10-01", "festival": "国庆大促"},
    {"date": "2024-11-11", "festival": "双11"},
    {"date": "2024-12-12", "festival": "双12"},
    {"date": "2024-12-25", "festival": "圣诞节"},

    {"date": "2025-01-29", "festival": "春节"},
    {"date": "2025-02-14", "festival": "情人节"},
    {"date": "2025-03-08", "festival": "女王节"},
    {"date": "2025-05-20", "festival": "520表白节"},
    {"date": "2025-06-18", "festival": "618年中大促"},
    {"date": "2025-07-29", "festival": "七夕节"},
    {"date": "2025-10-06", "festival": "中秋节"},
    {"date": "2025-10-01", "festival": "国庆大促"},
    {"date": "2025-11-11", "festival": "双11"},
    {"date": "2025-12-12", "festival": "双12"},
    {"date": "2025-12-25", "festival": "圣诞节"},
]

PROMO_DATES = [ pdate['date'] for pdate in promo_dates if pdate['festival'] in  ('618年中大促', '双11','双12', '圣诞节')]

HOLIDAY_DATES = [
    '2022-01-01','2022-02-01',  '2022-05-01', '2022-06-03', '2022-09-10', '2022-10-01', '2022-10-04', '2022-01-31',
    '2023-01-01', '2023-01-22',  '2023-05-01', '2023-06-22', '2023-09-29', '2023-10-01', '2023-10-23', '2023-01-21',
    '2024-01-01','2024-02-10',  '2024-05-01', '2024-06-10', '2024-09-17', '2024-10-01', '2024-10-02', '2024-02-09',
    '2025-01-01','2025-01-29',  '2025-05-01', '2025-06-20', '2025-10-06', '2025-10-01', '2025-09-30', '2025-01-28'
]

# 加载数据
df = load_data(FILEPATH, DATE_CUTOFF)
df['diff_0'] = 0
df = df[df.dim_ad_channel_level1 == '信息流']


df = add_date_features(df, PROMO_DATES, HOLIDAY_DATES)
# 特征处理
categorical_features = ['month','weekday','is_holiday','is_promo','season']
dense_features = ['install_deviceid_cnt','day1_ltv_sum','days_to_next_promo','days_past_last_promo']


# label处理
target = 'day7_ltv_label'
row_label = 'day7_ltv_sum'
diff_label = 'day1_ltv_sum'
df = process_label(df, row_label, diff_label, target)


In [20]:
df[['dt',row_label, diff_label, target]]

dt  day7_ltv_sum  day1_ltv_sum  day7_ltv_label
2    2021-12-31    963841.310    507017.000      456824.310
6    2022-01-01   1081194.030    558389.910      522804.120
10   2022-01-02   1129981.910    580690.450      549291.460
14   2022-01-03   1269620.590    559693.310      709927.280
18   2022-01-04   1186151.150    500054.140      686097.010
...         ...           ...           ...             ...
4890 2025-05-06   1801983.747   1061888.325      740095.422
4894 2025-05-07   1509906.493    973896.228      536010.265
4898 2025-05-08   1444136.247    937409.844      506726.403
4902 2025-05-09   1428940.536    927233.167      501707.369
4906 2025-05-10   1388785.108    907198.518      481586.590

[1227 rows x 4 columns]

In [21]:
df[df.dt.between('2025-02-01', '2025-02-10')]

dim_ad_channel_level1  install_deviceid_cnt  day1_ltv_sum  day2_ltv_sum   
4514                   信息流                 38257    857273.437   1099161.666  \
4518                   信息流                 46894    989238.735   1327731.012   
4522                   信息流                 46156   1180096.414   1516854.060   
4526                   信息流                 48666   1366489.981   1811360.277   
4530                   信息流                 52838   1595684.936   2037818.477   
4534                   信息流                 51523   1504802.279   1995961.472   
4538                   信息流                 53902   1447924.968   1862364.787   
4542                   信息流                 50815   1374618.125   1761481.532   
4546                   信息流                 54144   1308419.630   1669362.268   
4550                   信息流                 46679   1175872.262   1489366.307   

      day3_ltv_sum  day4_ltv_sum  day5_ltv_sum  day6_ltv_sum  day7_ltv_sum   
4514   1246810.187   1376139.385   1502104.824   1579340.208   1650630.305  \
4518   1562753.055   1733944.737   1855772.586   1938138.186   2009492.998   
4522   1720434.771   1840372.680   1944033.194   2037873.123   2091512.625   
4526   2010838.385   2190017.507   2297319.802   2375042.582   2446652.053   
4530   2276379.954   2378800.721   2475142.854   2563881.527   2631186.586   
4534   2179511.074   2320783.462   2407842.775   2513470.523   2578704.176   
4538   2045571.409   2185310.464   2292873.030   2340295.239   2432801.892   
4542   1943883.822   2064059.259   2159127.638   2232817.475   2310312.151   
4546   1839008.997   1948059.410   2050834.379   2149422.379   2248672.312   
4550   1622408.078   1716029.171   1836844.002   1892789.017   1936533.379   

      day30_ltv_sum  ...         dt  diff_0  month  weekday is_holiday   
4514    2177572.720  ... 2025-02-01       0      2        5          0  \
4518    2700179.771  ... 2025-02-02       0      2        6          0   
4522    2703514.800  ... 2025-02-03       0      2        0          0   
4526    3085191.970  ... 2025-02-04       0      2        1          0   
4530    3388499.061  ... 2025-02-05       0      2        2          0   
4534    3332178.756  ... 2025-02-06       0      2        3          0   
4538    3155903.066  ... 2025-02-07       0      2        4          0   
4542    3075359.397  ... 2025-02-08       0      2        5          0   
4546    3005344.128  ... 2025-02-09       0      2        6          0   
4550    2573836.818  ... 2025-02-10       0      2        0          0   

      is_promo  days_to_next_promo  days_past_last_promo  season   
4514         0                   0                     0  winter  \
4518         0                   0                     0  winter   
4522         0                   0                     0  winter   
4526         0                   0                     0  winter   
4530         0                   0                     0  winter   
4534         0                   0                     0  winter   
4538         0                   0                     0  winter   
4542         0                   0                     0  winter   
4546         0                   0                     0  winter   
4550         0                   0                     0  winter   

      day7_ltv_label  
4514      793356.868  
4518     1020254.263  
4522      911416.211  
4526     1080162.072  
4530     1035501.650  
4534     1073901.897  
4538      984876.924  
4542      935694.026  
4546      940252.682  
4550      760661.117  

[10 rows x 24 columns]